In [57]:
import numpy as np
import pandas as pd
import sqlite3
import pandasql
import sqlalchemy
from sqlalchemy import orm

In [58]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
adult = pd.read_csv(url,header = None,index_col=False)


In [59]:
cols=['AGE', 
        'WORKCLASS', 
        'FNLWGT', 
        'EDUCATION', 
        'EDUCATION_NUM', 
        'MARITAL_STATUS', 
        'OCCUPATION', 
        'RELATIONSHIP', 
        'RACE', 
        'SEX', 
        'CAPITAL_GAIN', 
        'CAPITAL_LOSS', 
        'HOURS_PER_WEEK', 
        'NATIVE_COUNTRY', 
        'GT50_OR_LT50K']

adult.columns=cols
adult.head(5)

,AGE,WORKCLASS,FNLWGT,EDUCATION,EDUCATION_NUM,MARITAL_STATUS,OCCUPATION,RELATIONSHIP,RACE,SEX,CAPITAL_GAIN,CAPITAL_LOSS,HOURS_PER_WEEK,NATIVE_COUNTRY,GT50_OR_LT50K
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


# 1. Create an sqlalchemy engine using a sample from the data set

In [60]:
engine = sqlalchemy.create_engine('sqlite:///:memory:',echo=False)
conn = engine.connect()

In [61]:
engine.execute('''
    CREATE TABLE IF NOT EXISTS ADULTS (
         AGE             INTEGER,
         WORKCLASS       VARCHAR(100),
         FNLWGT          INTEGER,
         EDUCATION       VARCHAR(100),
         EDUCATION_NUM   INTEGER,         
         MARITAL_STATUS  VARCHAR(100),         
         OCCUPATION      VARCHAR(100),
         RELATIONSHIP    VARCHAR(100),
         RACE            VARCHAR(100),
         SEX             VARCHAR(20),
         CAPITAL_GAIN    INTEGER,
         CAPITAL_LOSS    INTEGER,
         HOURS_PER_WEEK  INTEGER,
         NATIVE_COUNTRY  VARCHAR(100),
         GT50_OR_LT50K   VARCHAR(20))
''')

In [62]:
adult_sample = adult


In [63]:
dict_adult = adult_sample.to_dict(orient='records')

In [64]:
metadata = sqlalchemy.schema.MetaData(bind=engine)

In [65]:
adult_tab = sqlalchemy.Table('ADULTS',metadata,autoload=True)

In [66]:
Session = orm.sessionmaker(bind=engine)
session = Session()

In [67]:
conn.execute(adult_tab.insert(),dict_adult)

In [68]:
session.commit()

In [69]:
session.close()

In [70]:
sql_select = "SELECT * FROM ADULTS LIMIT 5"
conn = engine

In [71]:
result_adult = pd.read_sql_query(sql_select,conn)
result_adult

,AGE,WORKCLASS,FNLWGT,EDUCATION,EDUCATION_NUM,MARITAL_STATUS,OCCUPATION,RELATIONSHIP,RACE,SEX,CAPITAL_GAIN,CAPITAL_LOSS,HOURS_PER_WEEK,NATIVE_COUNTRY,GT50_OR_LT50K
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


# 2. Write two basic update queries

In [72]:
sql_update1 = " UPDATE ADULTS SET MARITAL_STATUS='Married' where MARITAL_STATUS like ' Married%';"

In [73]:
print(sql_update1)
engine.execute(sql_update1)

 UPDATE ADULTS SET MARITAL_STATUS='Married' where MARITAL_STATUS like ' Married%';


In [74]:
sql_update2 = " UPDATE ADULTS SET NATIVE_COUNTRY='Not Known' where NATIVE_COUNTRY =' ?';"
print(sql_update2,'\n\n')
engine.execute(sql_update2)

 UPDATE ADULTS SET NATIVE_COUNTRY='Not Known' where NATIVE_COUNTRY =' ?'; 




In [75]:
sql_select = "SELECT * FROM ADULTS WHERE (NATIVE_COUNTRY='Not Known') OR (MARITAL_STATUS='Married') LIMIT 5"

In [76]:
conn = engine

In [77]:
result_adult_data = pd.read_sql_query(sql_select,conn)
result_adult_data

,AGE,WORKCLASS,FNLWGT,EDUCATION,EDUCATION_NUM,MARITAL_STATUS,OCCUPATION,RELATIONSHIP,RACE,SEX,CAPITAL_GAIN,CAPITAL_LOSS,HOURS_PER_WEEK,NATIVE_COUNTRY,GT50_OR_LT50K
0,50,Self-emp-not-inc,83311,Bachelors,13,Married,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,53,Private,234721,11th,7,Married,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
2,28,Private,338409,Bachelors,13,Married,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
3,37,Private,284582,Masters,14,Married,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
4,49,Private,160187,9th,5,Married,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K


# 3. Write two delete queries

In [78]:
sql_delete = "DELETE FROM ADULTS WHERE OCCUPATION = ' ?' ;"
print(sql_delete,'\n\n')

sql_select = "SELECT * FROM ADULTS WHERE OCCUPATION = ' ?';"
conn = engine

result_adult = pd.read_sql_query(sql_select,conn)
print('No of records where OCCUPATION=\' ?\' before delete:\t',\
      result_adult_data['OCCUPATION'].count())

sql_select = "SELECT * FROM ADULTS;"
engine.execute(sql_delete)
conn = engine

result_adult_data = pd.read_sql_query(sql_select,conn)
print('No of records where OCCUPATION=\' ?\' after delete:\t', \
      result_adult_data['OCCUPATION'][(result_adult_data.OCCUPATION==' ?')].count())

DELETE FROM ADULTS WHERE OCCUPATION = ' ?' ; 


No of records where OCCUPATION=' ?' before delete:	 5
No of records where OCCUPATION=' ?' after delete:	 0


In [55]:
print('2nd Delete Query:')
#form the Query
sql_delete = "DELETE FROM ADULTS WHERE WORKCLASS =' ?' ;"
print(sql_delete,'\n\n')

# Check no of records where WORKCLASS==' ?' before delete
sql_select="SELECT * FROM ADULTS WHERE WORKCLASS = ' ?';"
conn=engine
result_adult_data=pd.read_sql_query(sql_select, conn) 
print('No of records where WORKCLASS=\' ?\' before delete:\t', \
              result_adult_data['WORKCLASS'].count())

# Execute the Delete Query
engine.execute(sql_delete)

# Check no of records where WORKCLASS==' ?' after delete
sql_select="SELECT * FROM ADULTS;"
conn=engine
result_adult_data=pd.read_sql_query(sql_select, conn) 

print('No of records where WORKCLASS=\' ?\' after delete:\t', \
      result_adult_data['WORKCLASS'][(result_adult_data.WORKCLASS==' ?')].count())

2nd Delete Query:
DELETE FROM ADULTS WHERE WORKCLASS =' ?' ; 


No of records where WORKCLASS=' ?' before delete:	 0
No of records where WORKCLASS=' ?' after delete:	 0


# 4. Write two filter queries

In [81]:
print('First query:')
sql_select = "SELECT * FROM ADULTS where AGE >= 80 AND EDUCATION = ' Masters';"
print(sql_select)
conn = engine
result_adult_data = pd.read_sql_query(sql_select,conn)
result_adult_data

First query:
SELECT * FROM ADULTS where AGE >= 80 AND EDUCATION = ' Masters';


,AGE,WORKCLASS,FNLWGT,EDUCATION,EDUCATION_NUM,MARITAL_STATUS,OCCUPATION,RELATIONSHIP,RACE,SEX,CAPITAL_GAIN,CAPITAL_LOSS,HOURS_PER_WEEK,NATIVE_COUNTRY,GT50_OR_LT50K
0,80,Private,157778,Masters,14,Widowed,Prof-specialty,Not-in-family,White,Female,0,0,10,United-States,<=50K
1,90,Local-gov,227796,Masters,14,Married,Exec-managerial,Husband,White,Male,20051,0,60,United-States,>50K
2,90,Private,51744,Masters,14,Never-married,Exec-managerial,Not-in-family,Black,Male,0,0,50,United-States,>50K
3,84,Private,241065,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Male,0,0,66,United-States,<=50K
4,81,Private,201398,Masters,14,Widowed,Prof-specialty,Unmarried,White,Male,0,0,60,Not Known,<=50K
5,90,Private,115306,Masters,14,Never-married,Exec-managerial,Own-child,White,Female,0,0,40,United-States,<=50K
6,90,Private,206667,Masters,14,Married,Prof-specialty,Wife,White,Female,0,0,40,United-States,>50K
7,86,Private,149912,Masters,14,Never-married,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,<=50K


In [82]:
print("Second filter query:")
sql_select = "SELECT * FROM ADULTS"
sql_select = sql_select+ " WHERE SEX=' Female' "
sql_select = sql_select+ " AND MARITAL_STATUS= ' Never-married' "
sql_select = sql_select+ " AND WORKCLASS != ' Private'"
sql_select = sql_select+ " AND AGE<18;"

print(sql_select)

conn = engine
result_adult_data = pd.read_sql_query(sql_select,conn)
result_adult_data

Second filter query:
SELECT * FROM ADULTS WHERE SEX=' Female'  AND MARITAL_STATUS= ' Never-married'  AND WORKCLASS != ' Private' AND AGE<18;


,AGE,WORKCLASS,FNLWGT,EDUCATION,EDUCATION_NUM,MARITAL_STATUS,OCCUPATION,RELATIONSHIP,RACE,SEX,CAPITAL_GAIN,CAPITAL_LOSS,HOURS_PER_WEEK,NATIVE_COUNTRY,GT50_OR_LT50K
0,17,Local-gov,182070,11th,7,Never-married,Other-service,Own-child,White,Female,0,0,16,United-States,<=50K
1,17,Local-gov,246308,11th,7,Never-married,Prof-specialty,Own-child,White,Female,0,0,20,Puerto-Rico,<=50K
2,17,Local-gov,148194,11th,7,Never-married,Adm-clerical,Own-child,White,Female,0,0,12,United-States,<=50K
3,17,Self-emp-not-inc,228786,10th,6,Never-married,Other-service,Own-child,White,Female,0,0,24,United-States,<=50K
4,17,Local-gov,308901,11th,7,Never-married,Adm-clerical,Own-child,White,Female,0,0,15,United-States,<=50K
5,17,Self-emp-inc,413557,9th,5,Never-married,Sales,Own-child,White,Female,0,0,40,United-States,<=50K
6,17,Local-gov,244856,11th,7,Never-married,Prof-specialty,Own-child,White,Female,0,0,40,United-States,<=50K
7,17,Local-gov,170916,10th,6,Never-married,Protective-serv,Own-child,White,Female,0,1602,40,United-States,<=50K
8,17,Local-gov,340043,12th,8,Never-married,Adm-clerical,Own-child,White,Female,0,0,12,United-States,<=50K
9,17,Federal-gov,99893,11th,7,Never-married,Adm-clerical,Not-in-family,Black,Female,0,1602,40,United-States,<=50K


# 5. Write two function queries

In [84]:
print('First function query:')
sql_select = "SELECT COUNT(*) AS COUNT_AGE_GT_60 FROM ADULTS WHERE AGE >= 60;"
print(sql_select)
conn = engine
result_adult_data = pd.read_sql_query(sql_select,conn)
result_adult_data

First function query:
SELECT COUNT(*) AS COUNT_AGE_GT_60 FROM ADULTS WHERE AGE >= 60;


,COUNT_AGE_GT_60
0,2120


In [86]:
print("Second function query:")
sql_select = "SELECT SEX,COUNT(*) AS COUNT_GENDER FROM ADULTS GROUP BY SEX;"
print(sql_select)
conn = engine
result_adult_data = pd.read_sql_query(sql_select,conn)
result_adult_data

Second function query:
SELECT SEX,COUNT(*) AS COUNT_GENDER FROM ADULTS GROUP BY SEX;


,SEX,COUNT_GENDER
0,Female,9930
1,Male,20788
